In [ ]:
import ray 
import numpy as np
import logging
import time

import warnings
warnings.filterwarnings('ignore') 

Stime = time.perf_counter()

# Initialize the logger
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

# Function that runs a computation on CPU
@ray.remote(num_cpus=1)
def perform_cpu_task():
    try:
        # Check if CPU is available and being utilized
        if ray.get_gpu_ids():
            # Perform metric multiplication on the CPU using NumPy
            a = np.array([[1, 2], [3, 4]], dtype=np.float32)
            b = np.array([[5, 6], [7, 8]], dtype=np.float32)
            result = np.matmul(a, b)
            return result
        else:
            logger.error("GPU not available.")
            return None
    except Exception as e:
        logger.error(f"Error in GPU task: {e}")
        return None
    
@ray.remote(num_cpus=1)
def create_matrix(size):
    return np.random.normal(size=size)

@ray.remote(num_cpus=1)
def multiply_matrices(x, y):
    return np.dot(x, y)

@ray.remote(num_cpus=1)
def sum_matrices(x, y):
    return np.add(x, y)

print(f"Finished in {time.perf_counter()-Stime:.2f}")

def main():
    try:
        # Initialize Ray
        ray.init(address="ray://kuberay-head-svc.kuberay:10001", ignore_reinit_error=True)
        
        print("=============:", ray.cluster_resources())

        # Call the remote function on GPU
        m1 = create_matrix.remote([10000, 10000])
        m2 = create_matrix.remote([10000, 10000])
        m3 = create_matrix.remote([10000, 10000])
        m4 = create_matrix.remote([10000, 10000])

        m12 = multiply_matrices.remote(m1, m2)
        m34 = multiply_matrices.remote(m3, m4)

        a12_34 =  sum_matrices.remote(m12, m34)

        ## Results
        MM = ray.get(a12_34)
        result = ray.get(perform_cpu_task.remote())
        
        if result is not None:
            logger.info("Metric Multiplication Result on CPU:")
            logger.info(result)
            logger.info(MM)
        else:
            logger.error("CPU task failed.")
    except Exception as e:
        logger.error(f"Error in main: {e}")
    finally:
        # Shutdown Ray gracefully
        ray.shutdown()

if __name__ == "__main__":
    main()


2023-07-30 15:08:26,825	INFO client_builder.py:252 -- Passing the following kwargs to ray.init() on the server: ignore_reinit_error


Finished in 0.00
=============: {'node:10.244.0.164': 1.0, 'node:10.244.1.154': 1.0, 'object_store_memory': 4673303347.0, 'memory': 16000000000.0, 'CPU': 3.0}


(raylet, ip=10.244.1.154) Spilled 5340 MiB, 7 objects, write throughput 47 MiB/s.
